# GUI using Tkinter

In [8]:
import tkinter
from tkinter import *

def Classify_problem(msg):
    # importing libraries
    import pandas as pd
    import numpy as np
    from sklearn import svm
    from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    from nltk import FreqDist, pos_tag
    from nltk.corpus import wordnet
    import string
    import re
    import pickle
    # text preprocessing
    tag_dict = {"J": wordnet.ADJ,"N": wordnet.NOUN,"V": wordnet.VERB,"R": wordnet.ADV}
    text = msg.replace('\d+', '')
    text = text.replace('[^\w\s]', '')
    text = text.lower()
    lemmatizer = WordNetLemmatizer()
    text=[lemmatizer.lemmatize(item,tag_dict.get(pos_tag([item])[0][1][0].upper(),wordnet.NOUN)) for item in word_tokenize(text)]
    text = ' '.join(text)

  # pickle files
    Count_Vect_ = pickle.load(open("count_vect_svm.pkl", "rb"))
    Transformer_ = pickle.load(open("transformer_svm.pkl", "rb"))
    SVM_tf = pickle.load(open("model_svm.pkl", "rb"))

    
#   Predict Probabilities With tfidf-SVM,RF
    counts = Count_Vect_.transform([text])
    counts = Transformer_.transform(counts)
    tfidf_svm= SVM_tf.predict_proba(counts)
    cls7=['Anger Management' ,'Behavioral Change', 'Counseling Fundamentals',
         'Depression' ,'Family Conflict' ,'Intimacy', 'LGBTQ',
          'Marriage', 'Parenting', 'Relationship Dissolution', 'Relationships',
        'Self-esteem' ,'Sleep Improvement' ,'Social Relationships' ,'Stress/Anxiety',
         'Substance Abuse/Addiction' ,'Trauma/Grief/Loss', 'Workplace issues']
    maxpos = tfidf_svm.argmax()
    label=cls7[maxpos]
    return label
def Vader_sentiment_scores(sentence):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] > 0.2 :
        return("Positive")
    else :
        return("Negative")  
def similarity_test(query):
    import scipy
    from sentence_transformers import SentenceTransformer
    input_msg=[query]
    model=SentenceTransformer('bert-base-nli-mean-tokens')
    input_embeddings= model.encode(input_msg)
    queries=['How can I control myself and my anger?',
       'How do I not be angry all the time?',
       'How can I control my temper?', 'Why do I get angry so easily?',
       'How do I deal with my alcoholic boyfriend with a dark past?',
       'My girlfriend always brings up past events and talks negatively about them',
       'Why did my boyfriend hit himself in the face during an argument?',
       'How can I just be happy and not mad all the time?',
       'How can I be less angry?', 'I need help controlling my anger',
       "How do I deal with my son's violent thoughts and dreams?",
       "How can I deal with the anger problems I've gained from my soon-to-be husband?",
       'How can I stop being so angry?', 'Why am I so mad?',
       'How can I control myself and learn to let things go or communicate?',
       'How can I be less confused about my feelings towards anything?',
       'I need to know how to cope with misophonia before I go completely insane',
       'i need answers to my angry, possessiveness, and urges',
       'How can I control my anger?', 'How do I manage my anger?',
       'Why am I constantly angry?',
       'My husband and I canâ€™t talk to each other without arguing',
       'Why do I get random spurts of anger over petty things?',
       'I have anger issues. I am extremely explosive about the simplest things']
    query_embeddings=model.encode(queries)
    closest_n=1
    dist_scores=[]
    for query,query_embedding in zip(queries,query_embeddings):
        distances=scipy.spatial.distance.cdist([query_embedding],input_embeddings,"cosine")[0]
        score=(1-distances)
        dist_scores.extend(score)
    return max(dist_scores)

def chatbot_response(msg):
    import pandas as pd
    msg=msg.lower()
    sentiment=Vader_sentiment_scores(msg)
    greetings = ['hi','hey', 'hello', 'heyy', 'hi', 'hey', 'good evening', 'good morning', 'good afternoon', 'good', 'fine', 'okay', 'great', 'could be better', 'not so great', 'very well thanks', 'fine and you', "i'm doing well", 'pleasure to meet you', 'hi whatsup']
    goodbyes = ['thank you', 'thank you', 'yes bye', 'bye', 'thanks and bye', 'ok thanks bye', 'goodbye', 'see ya later', 'alright thanks bye', "that's all bye", 'nice talking with you', 'i’ve gotta go', 'i’m off', 'good night', 'see ya', 'see ya later', 'catch ya later', 'adios', 'talk to you later', 'bye bye', 'all right then', 'thanks', 'thank you', 'thx', 'thx bye', 'thnks', 'thank u for ur help', 'many thanks', 'you saved my day', 'thanks a bunch', "i can't thank you enough", "you're great", 'thanks a ton', 'grateful for your help', 'i owe you one', 'thanks a million', 'really appreciate your help', 'no', 'no goodbye']
    if msg in greetings:
        answer="Hi! I\'m a Mental Health chatbot!.My name is Manas.Please ask me for help whenever you feel like it! I\'m always online."
    elif msg in goodbyes:
        answer="Hope I was able to help you today! Take care, bye!"
    elif sentiment=="Positive":
        answer="That's great! Do you still have any questions for me?"
    else:
        category=Classify_problem(msg)
        import pandas as pd
        df=pd.read_excel("C:\\Users\\Garlapati.Varun\\Downloads\\Manas\\Answers_short.xlsx")
        df = df[df['Topic'] == category]
        query=df['comp'].tolist()
        scores=[]
        for queries in query:
            a=similarity_test(queries)
            scores.append(a)
        df['scores']=scores
        df=df[df['scores']==df['scores'].max()]
        answer_out=df['Summarized answer'].tolist()
        answer_out.insert(0, "Tough to hear that.Here is my soloution")
        answer = ''.join(answer_out)
    return answer

def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Manas-Mental Health Chatbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=3,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="3", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()